# Assignment 3 - Data Cleaning and Analysis

이번 과제에서 학생들은 주어진 데이터를 clean up 하고 분석하는 연습을 하게 될 것이다.

두개의 지정된 질문에 대한 답을 하여야 하고 세개의 질문을 스스로 만들어 코드와 함께 결과를 제시한다.

* 제출 방법: GitHub (10/21 자정까지)
* 파일 이름: A3-학번-이름(영어로).iphynb 형식으로 (예: A3-13403-999-jiyoon.iphynb)

### Dataset

타이타닉호의 모든 승객, 승무원의 정보와 생존자, 사망자 정보는 다음의 웹사이트에 잘 정리되어있다.
* http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic.html

본 과제에서 제시하는 데이터는 해당 웹사이트의 정보를 담고 있다. 다음의 데이터를 다운로드하여 분석하고 아래의 질문에 답하시오.
* 데이터: http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls
* 데이터 설명: http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic.html
* 데이터 변수 설명: http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3info.txt

### Q1: 생존자와 사망자의 평균 나이를 남녀를 구분하여 계산하시오.

In [1]:
import pandas as pd
import numpy as np

titanic = pd.read_csv("titanic3.csv")

# 1. 먼저 filtering을 해서 생존자와 사망자로 구분한다.
survived = titanic[titanic.survived == 1]
dead = titanic[titanic.survived == 0]

# 2. 데이터를 살펴보니 Age에 NaN으로 표기되어 있는 사람들이 있다. 얼마나 있는지 살펴보자.
survived_empty = survived.apply(lambda col: pd.isnull(col))
dead_empty = dead.apply(lambda col: pd.isnull(col))
count_s = 0
count_d = 0
for i in range(len(survived_empty)):
    if survived_empty.age.iloc[i] == True:
        count_s += 1
for i in range(len(dead_empty)):
    if dead_empty.age.iloc[i] == True:
        count_d += 1
# 성별에도 missing data가 있는지 살펴본다
count_sex_survived = 0
count_sex_dead = 0
for i in range(len(survived_empty)):
    if survived_empty.sex.iloc[i] == True:
        count_sex_survival += 1
for i in range(len(dead_empty)):
    if dead_empty.sex.iloc[i] == True:
        count_sex_dead += 1
count_sex = count_sex_survived + count_sex_dead
# print(count_sex) # == 0
# print(count_s) # == 73
# print(count_d) # == 190
# 현재 생존자 데이터에서는 73개의 나이가 빠져있고, 사망자 데이터에서는 190개의 나이가 빠져있다. 다행히 성별정보는 모두 있다.

# 3. 따라서 이 분실된 데이터를 처리하기 위해 분실된 데이터에 빈 값을 채워넣는다
survived.age = survived.age.fillna("")
dead.age = dead.age.fillna("")

# 4. 이 상태에서 각각의 데이터를 다시 남녀로 구분한다.
survived_male = survived[survived.sex == "male"]
survived_female = survived[survived.sex == "female"]
dead_male = dead[dead.sex == "male"]
dead_female = dead[dead.sex == "female"]

# 5. 각각의 데이터프레임에서 나이의 평균값을 내는 함수를 만든다. 단, 이때 age값이 0인 사람은 계산에서 배제한다
def mean_age(col):
    null_count = 0 # 빈 데이터의 개수를 세기 위한 변수
    age_sum = 0 # 나이의 합을 구하기 위한 변수
    for i in range(len(col)):
        # 만일 칼럼의 값이 비어있다면 빈 데이터에 개수에 1개씩 추가해라
        if col.iloc[i] == "":
            null_count += 1
        # 만일 칼럼의 값이 비어있지 않다면 그 값을 나이합에 합하라
        else:
            age_sum += col.iloc[i]
    # 데이터가 실제로 있는 사람들의 나이 평균을 구해서 소수점 셋째 자리에서 반올림
    mean = round(float(age_sum / (len(col) - null_count)), 2)
    return mean

print(mean_age(survived_male.age))
print(mean_age(survived_female.age))
print(mean_age(dead_male.age))
print(mean_age(dead_female.age))

26.98
29.82
31.52
25.26


c:\users\wltjd\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


### Q2: 생존자의 평균 티켓 가격과 사망자의 평균 티켓 가격을 구하고 비교하시오.

In [2]:
# 1. 이번엔 각각의 데이터프레임에서 티켓 가격이 비어 있는 항목이 있는지 살펴본다
count_fs = 0
count_fd = 0
for i in range(len(survived_empty)):
    if survived_empty.fare.iloc[i] == True:
        count_fs += 1
for i in range(len(dead_empty)):
    if dead_empty.fare.iloc[i] == True:
        count_fd += 1
# print(count_fs) # == 0
# print(count_fd) # == 1
# 현재 사망자 데이터에 티켓값이 비어있는 사람이 1명 있다.
dead_empty[dead_empty.fare == True]
# print(dead.loc[1225])

# 2. 현재 1225번째 승객의 운임이 빠져있다. 따라서 이 사람의 fare 데이터를 채워준다
dead.fare = dead.fare.fillna("")
dead.loc[1225]

# 3. 각각의 데이터프레임에 대해 평균 티켓값을 구하는 함수를 만든다.
def mean_fare(col):
    null_count = 0 # 빈 데이터의 개수를 세기 위한 변수
    fare_sum = 0 # 운임의 합을 구하기 위한 변수
    for i in range(len(col)):
        # 만일 칼럼의 값이 비어있다면 빈 데이터에 개수에 1개씩 추가해라
        if col.iloc[i] == "":
            null_count += 1
        # 만일 칼럼의 값이 비어있지 않다면 그 값을 나이합에 합하라
        else:
            fare_sum += col.iloc[i]
    # 데이터가 실제로 있는 사람들의 운임 평균을 구해서 소수점 셋째 자리에서 반올림
    mean = round(float(fare_sum / (len(col) - null_count)), 2)
    return mean

print(mean_fare(survived.fare))
print(mean_fare(dead.fare))

49.36
23.35


### Q3: 혼자 탑승한 승객은 어떤 class에 많이 탑승했는지 알아보고, class별 생존자 비율을 구하시오

In [3]:
# 혼자 탑승한 승객만 filtering
single = titanic[(titanic.parch == 0) & titanic.sibsp == 0]

# 1. 먼저 class 데이터에 없는 값이 있는지 살펴본다
empty_single = single.apply(lambda col: pd.isnull(col))
null_pclass = 0
for i in range(len(empty_single)):
    if empty_single.pclass.iloc[i] == True:
        null_pclass += 1
# print(null_pclass) # == 1 
# class값이 비어있는 곳이 있으므로 찾아본다
empty_single[empty_single.pclass == True]
# 위의 데이터는 아예 비어있는 row이므로 삭제한다
single = single.dropna(how='all') # 데이터가 전부 비어있는 row를 삭제하는 메소드
single
# 즉 class 데이터는 모두 들어차있다

# 2. 어떤 class에 많이 탑승했는지 알아본다.
single_class1 = 0
single_class2 = 0
single_class3 = 0
for i in range(len(single)):
    if single.pclass.iloc[i] == 1:
        single_class1 += 1
    elif single.pclass.iloc[i] == 2:
        single_class2 += 1
    else:
        single_class3 += 1
print(single_class1)
print(single_class2)
print(single_class3)
# print(single_else)

# 3. 혼자 탑승한 승객의 class별 생존자 비율을 구한다
single_class1_survived = 0
single_class2_survived = 0
single_class3_survived = 0
for i in range(len(single)):
    if single.pclass.iloc[i] == 1 and single.survived.iloc[i] == 1:
        single_class1_survived += 1
    elif single.pclass.iloc[i] == 2 and single.survived.iloc[i] == 1:
        single_class2_survived += 1
    elif single.pclass.iloc[i] == 3 and single.survived.iloc[i] == 1:
        single_class3_survived += 1

def ratio(a, b):
    ratio = round((float(a / b) * 100), 2)
    print(str(ratio) + "%")

print("")
ratio(single_class1_survived, single_class1)
ratio(single_class2_survived, single_class2)
ratio(single_class3_survived, single_class3)

241
229
627

58.51%
44.98%
25.36%


### Q4: 가족승객 가운데, class별 평균 동반 인원수를 구하시오.

In [4]:
# 가족 승객만 따로 filtering한다
family = titanic[(titanic.parch != 0) | (titanic.sibsp != 0)]

# 1. parch와 sibsp 각각의 column에 빈값이 있는지 살펴본다
empty_family = family.apply(lambda col: pd.isnull(col))
count = 0
for i in range(len(empty_family)):
    if (empty_family.sibsp.iloc[i] == True) or (empty_family.parch.iloc[i] == True):
        count += 1
# print(count)
empty_family[(empty_family.sibsp == True) | (empty_family.parch == True)]
# 전체가 비어있는 행이므로 삭제한다
family = family.dropna(how='all')
family

# 2. 이 데이터를 다시 class별로 필터링한다
family_class1 = family[family.pclass == 1]
family_class2 = family[family.pclass == 2]
family_class3 = family[family.pclass == 3]

# 3. 동반 인원수 계산
def mean_accompany(data, col1, col2):
    accompany_1 = 0
    accompany_2 = 0
    for i in range(len(col1)):
        accompany_1 += col1.iloc[i]
    for i in range(len(col2)):
        accompany_2 += col2.iloc[i]
    sum_accompany = accompany_1 + accompany_2
    mean_accompany = round(float(sum_accompany / len(data)), 2)
    return mean_accompany

print(mean_accompany(family_class1, family_class1.sibsp, family_class1.parch))
print("")
print(mean_accompany(family_class2, family_class2.sibsp, family_class2.parch))
print("")
print(mean_accompany(family_class3, family_class3.sibsp, family_class3.parch))

1.59

1.77

2.9


### Q5: 각각의 승선항에서 어떤 class의 사람이 많이 탔는지 그 비율과, 그들의 평균 운임을 구하시오.

In [11]:
# 1. 각각의 클래스 데이터와 운임 데이터에서 빈 항목이 있는지 살펴본다
empty_titanic = titanic.apply(lambda col: pd.isnull(col))
def count_empty(data, col):
    count_empty = 0
    for i in range(len(data)):
        if (col.iloc[i] == True):
            count_empty += 1
    return count_empty
# empty_titanic[(empty_titanic.pclass == True) | (empty_titanic.fare == True)]
# 현재 1225번째 승객과 1309번째 승객의 데이터에서 각각 fare와 pclass 데이터가 빠져있다.
# 다만 1309번째 승객은 빈 데이터이므로 삭제하는 것이 낫고, 1225번째 승객은 fare에 빈값만 채워준다
titanic = titanic.dropna(how='all')
titanic = titanic.fillna("")
titanic

# 2. 이렇게 수정된 데이터를 다시 승선항별로 filtering한다. 이때 C = Cherbourg; Q = Queenstown; S = Southampton이다
C = titanic[titanic.embarked == "C"]
Q = titanic[titanic.embarked == "Q"]
S = titanic[titanic.embarked == "S"]
# print(len(C))
# print(len(Q))
# print(len(S))

# 3. 승선항별 각각의 class에 탄 사람들의 비율을 구한다.
def class_count(data, number):
    # 각각의 데이터에 대해서 데이터 항목의 개수가 다르기 때문에 데이터 역시 매개변수로 넣어주어야 함
    class_count = 0
    for i in range(len(data)):
        if data.pclass.iloc[i] == number:
            class_count += 1
    ratio = round(float(class_count / len(data)) * 100, 2)
    return ratio

def class_count_ratio(index):
    class_count_ratio = {}
    for i in range(1,4):
        if i == 1:
            class_count_ratio[str(i) + "st class"] = str(class_count(index, i)) + "%"
        elif i == 2:
            class_count_ratio[str(i) + "nd class"] = str(class_count(index, i)) + "%"
        else:
            class_count_ratio[str(i) + "rd class"] = str(class_count(index, i)) + "%"
    return class_count_ratio

# 4. 승선항별로 사람들의 운임 평균을 구한다.
def fare_mean(data):
    fare_sum = 0
    null_count = 0
    for i in range(len(data)):
        if data.fare.iloc[i] == "":
            null_count += 1
        else:
            fare_sum += data.fare.iloc[i]
    fare_mean = round(float(fare_sum / (len(data) - null_count)), 2)
    return fare_mean

print("Queensland에서 탄 사람의 class별 비율과 평균운임은 다음과 같다")
print(class_count_ratio(Q))
print(fare_mean(Q))
print("")
print("Cherbourg에서 탄 사람의 class별 비율과 평균운임은 다음과 같다")
print(class_count_ratio(C))
print(fare_mean(C))
print("")
print("Southampton에서 탄 사람의 class별 비율과 평균운임은 다음과 같다")
print(class_count_ratio(S))
print(fare_mean(S))

Queensland에서 탄 사람의 class별 비율과 평균운임은 다음과 같다
{'1st class': '2.44%', '2nd class': '5.69%', '3rd class': '91.87%'}
12.41

Cherbourg에서 탄 사람의 class별 비율과 평균운임은 다음과 같다
{'1st class': '52.22%', '2nd class': '10.37%', '3rd class': '37.41%'}
62.34

Southampton에서 탄 사람의 class별 비율과 평균운임은 다음과 같다
{'1st class': '19.37%', '2nd class': '26.48%', '3rd class': '54.16%'}
27.42
